In [2]:
def train_estimator():
    """
    Función principal que entrena un modelo de clasificación de texto usando regresión logística.
    Esta función implementa un pipeline completo de procesamiento de texto y entrenamiento.
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍAS NECESARIAS
    # ============================================================================
    import pandas as pd  # Para manejo de datos estructurados (DataFrames)
    from sklearn.feature_extraction.text import TfidfTransformer  # Para transformar conteos de palabras a pesos TF-IDF
    from sklearn.linear_model import LogisticRegression  # Algoritmo de clasificación por regresión logística
    from sklearn.model_selection import train_test_split  # Para dividir datos en entrenamiento y prueba
    from sklearn.pipeline import Pipeline  # Para crear un flujo de procesamiento secuencial
    from sklearn.metrics import balanced_accuracy_score  # Métrica de evaluación balanceada
    from sklearn.feature_extraction.text import CountVectorizer  # Para convertir texto a matriz de conteos de palabras
    from sklearn.metrics import accuracy_score, balanced_accuracy_score  # Métricas adicionales de evaluación
    # import os  # Para operaciones del sistema operativo (comentado, no se usa)
    import pickle  # Para serializar y guardar el modelo entrenado

    # ============================================================================
    # CARGA Y PREPARACIÓN DE LOS DATOS
    # ============================================================================
    
    # Cargar el dataset desde un archivo CSV comprimido
    # El archivo contiene frases de texto y sus etiquetas de clasificación
    dataframe = pd.read_csv(
        "../files/input/sentences.csv.zip",  # Ruta al archivo comprimido
        index_col=False,  # No usar ninguna columna como índice
        compression="zip",  # Especificar que el archivo está comprimido en ZIP
    )

    # Separar las características (texto) de las etiquetas (target)
    data = dataframe.phrase  # Columna con las frases de texto a clasificar
    target = dataframe.target  # Columna con las etiquetas verdaderas (clases)
    
    # ============================================================================
    # DIVISIÓN DE DATOS EN ENTRENAMIENTO Y PRUEBA
    # ============================================================================
    
    # Dividir el dataset en conjuntos de entrenamiento (70%) y prueba (30%)
    X_train, X_test, y_train, y_test = train_test_split(
        data,  # Datos de entrada (frases)
        target,  # Etiquetas objetivo
        test_size=0.3,  # 30% para prueba, 70% para entrenamiento
        shuffle=False,  # Mantener el orden original (importante para datos temporales)
    )

    # ============================================================================
    # CONFIGURACIÓN DEL PIPELINE DE PROCESAMIENTO DE TEXTO
    # ============================================================================
    
    # Paso 1: Vectorización del texto
    # Convierte texto crudo en una matriz numérica de conteos de palabras
    vectorizer = CountVectorizer(
        lowercase=True,  # Convertir todo el texto a minúsculas para normalización
        analyzer="word",  # Analizar a nivel de palabras (no caracteres)
        token_pattern=r"\b[a-zA-Z]\w+\b",  # Patrón regex: palabras que empiecen con letra y contengan caracteres alfanuméricos
        stop_words="english",  # Filtrar palabras comunes en inglés (the, and, is, etc.)
    )

    # Paso 2: Transformación TF-IDF
    # Convierte los conteos brutos en pesos TF-IDF que dan más importancia a palabras discriminativas
    transformer = TfidfTransformer()

    # ============================================================================
    # CREACIÓN Y CONFIGURACIÓN DEL PIPELINE COMPLETO
    # ============================================================================
    
    # Crear un pipeline que ejecuta secuencialmente:
    # 1. Vectorización del texto → 2. Transformación TF-IDF → 3. Clasificación
    lr_estimator = Pipeline(
        steps=[
            ("vectorizer", vectorizer),  # Primer paso: convertir texto a matriz de conteos
            ("transformer", transformer),  # Segundo paso: aplicar transformación TF-IDF
            ("estimator", LogisticRegression(max_iter=1000)),  # Tercer paso: entrenar clasificador
        ],
        verbose=False,  # No mostrar información detallada durante el entrenamiento
    )

    # ============================================================================
    # ENTRENAMIENTO DEL MODELO
    # ============================================================================
    
    # Entrenar el pipeline completo con los datos de entrenamiento
    # Esto ejecuta automáticamente todos los pasos del pipeline en secuencia
    lr_estimator.fit(X_train, y_train)

    # ============================================================================
    # PERSISTENCIA DEL MODELO ENTRENADO
    # ============================================================================
    
    # Guardar el modelo entrenado en un archivo binario para uso posterior
    # Esto permite reutilizar el modelo sin necesidad de re-entrenarlo
    with open("estimator.pickle", "wb") as file:
        pickle.dump(lr_estimator, file)  # Serializar y guardar el pipeline completo

# ============================================================================
# EJECUCIÓN DE LA FUNCIÓN DE ENTRENAMIENTO
# ============================================================================

# Llamar a la función para ejecutar todo el proceso de entrenamiento
train_estimator()



In [3]:
def use_estimator():
    """
    Función que carga un modelo previamente entrenado y lo utiliza para hacer predicciones.
    Esta función implementa la fase de inferencia del pipeline de machine learning.
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍAS PARA INFERENCIA
    # ============================================================================
    import pickle  # Para deserializar y cargar el modelo guardado
    import pandas as pd  # Para manejo de datos estructurados

    # ============================================================================
    # CARGA DE DATOS PARA PREDICCIÓN
    # ============================================================================
    
    # Cargar el mismo dataset que se usó para entrenamiento
    # En un escenario real, estos serían datos nuevos no vistos durante el entrenamiento
    dataframe = pd.read_csv(
        "../files/input/sentences.csv.zip",  # Archivo con las frases a clasificar
        index_col=False,  # No usar ninguna columna como índice
        compression="zip",  # El archivo está comprimido en formato ZIP
    )

    # Extraer únicamente las frases de texto (no necesitamos las etiquetas para predicción)
    data = dataframe.phrase  # Columna con las frases que queremos clasificar

    # ============================================================================
    # CARGA DEL MODELO PREVIAMENTE ENTRENADO
    # ============================================================================
    
    # Cargar el pipeline completo desde el archivo donde fue guardado
    # Esto incluye el vectorizador, transformador TF-IDF y el clasificador entrenado
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)  # Deserializar el modelo completo

    # ============================================================================
    # GENERACIÓN DE PREDICCIONES
    # ============================================================================
    
    # Aplicar el modelo a los nuevos datos para obtener predicciones
    # El pipeline automáticamente:
    # 1. Vectoriza el texto usando el mismo vocabulario del entrenamiento
    # 2. Aplica la transformación TF-IDF con los mismos parámetros
    # 3. Genera predicciones usando el clasificador entrenado
    prediction = estimator.predict(data)

    # ============================================================================
    # RETORNO DE RESULTADOS
    # ============================================================================
    
    # Devolver las predicciones para que puedan ser utilizadas por otros procesos
    return prediction

# ============================================================================
# EJECUCIÓN DE LA FUNCIÓN DE PREDICCIÓN
# ============================================================================

# Llamar a la función para ejecutar el proceso de inferencia
# El resultado serán las predicciones de clase para cada frase en el dataset
use_estimator()

array(['neutral', 'positive', 'positive', ..., 'neutral', 'positive',
       'neutral'], shape=(2264,), dtype=object)